# Pré processamento

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 54.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=10c9c75c3228aed41b3833f29bc56ff9e25bcaa465c209cc0b25323e4528c4a3
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder\
                    .master("local[*]")\
                    .appName("Desafio Alura Data Science 2 - Semana 1 - Transformação de dados")

# Carga de Dado

In [4]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=98f45dfd0e40ee94304f038bcd19aca15422598cb9d4b7d14a4f497b61f13e78
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [6]:
!wget https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip && unzip semana-1.zip -d dados/

--2022-12-11 19:32:30--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.217.161.57, 52.216.27.116, 52.217.132.57, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.217.161.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip.1’

semana-1.zip.1      100%[===================>]  18.10M  7.58MB/s    in 2.4s    

2022-12-11 19:32:33 (7.58 MB/s) - ‘semana-1.zip.1’ saved [18975214/18975214]

Archive:  semana-1.zip
  inflating: dados/dataset_bruto.json  


In [7]:
dados = '/content/dados/dataset_bruto.json'

In [9]:
df = spark.getOrCreate().read.json(dados)

## Impressão do dado bruto

In [11]:
df.show(5)

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



## Quantidade de linhas e colunas

In [12]:
linhas = df.count()
colunas = len(df.columns)
print(f'O dado bruto apresenta {linhas} linhas e {colunas} colunas!')

O dado bruto apresenta 89083 linhas e 3 colunas!


# Avaliando a estrutura dos dados

In [13]:
df.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

##### Possivelmente os dados necessários para entrada no modelo estão na coluna 'anuncio'

In [14]:
df_anuncio = df.select('anuncio.*')
df_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [15]:
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

# Analise do tipo de uso dos imóveis

In [16]:
df_anuncio\
    .select('tipo_uso')\
    .groupby('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



#### A quantidade de imóveis comerciais é quase irrevelevante em relação aos residenciais, o modelo gerado pode não representar o comportamento destes. Portanto, os dados serão filtrados para seguir com os residenciais

In [17]:
df_anuncio = df_anuncio\
                    .filter(df_anuncio.tipo_uso == 'Residencial')

df_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|     [120]|    [120]|      [0]|[Portão eletrônic...|{Campo Grande, 23...|ac6cb348-69d6-45a...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{80, null, Venda...|
|    0|       [3]|      [3]|      [0]|                  []|{São Cristóvão, 2...|e032b908-ef42-4d4...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{0, 0, Venda, 50...|
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-4

In [18]:
df_anuncio.count()

84541

# Analise do tipo de unidades residenciais

In [19]:
df_anuncio\
    .select('tipo_unidade')\
    .groupby('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7517|
| Apartamento|66797|
|        Casa|10227|
+------------+-----+



#### A quantidade de apartamentos é muito superior aos demais tipos de unidades, portanto é recomendável seguir somente com essa classe

In [20]:
df_anuncio = df_anuncio\
                    .filter("tipo_unidade == 'Apartamento'")

df_anuncio.count()

66797

In [21]:
df_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

# Analisar a quantidade de tipo de anuncio

In [22]:
df_anuncio\
    .select('tipo_anuncio')\
    .groupby('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
|  Lançamento|  235|
+------------+-----+



#### Deve ser seguir com anuncios somente imóveis usados, dada a diferença na quantidade entre lançamentos e usados

In [23]:
df_anuncio = df_anuncio\
                    .filter("tipo_anuncio == 'Usado'")

In [24]:
df_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

#### Há features que estão em formato de array, para poderem ser utilizados os valores dentro de [ ] serão extraídos para a tabela

In [25]:
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [26]:
arrays = ['area_total', 'area_util', 'banheiros', 'quartos', 'suites', 'vaga']

In [27]:
df_anuncio\
    .select([f.col(c)[0].alias(c) if c in arrays else c for c in df_anuncio.columns])\
    .show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [28]:
df_anuncio = df_anuncio\
    .select([f.col(c)[0].alias(c) if c in arrays else c for c in df_anuncio.columns])

# Transformação dos dados extraídos das arrays para inteiro

In [29]:
df_anuncio\
    .select([f.col(c).cast(IntegerType()) if c in arrays else c for c in df_anuncio.columns])\
    .printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 

In [30]:
df_anuncio = df_anuncio\
                .select([f.col(c).cast(IntegerType()) if c in arrays else c for c in df_anuncio.columns])

# Extração de informação de localização (bairro e zona)

In [32]:
df_anuncio\
  .select('*', 'endereco.bairro', 'endereco.zona')\
  .drop('endereco')\
  .show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|Paciência|Zona Oeste|
|    1|        41|       41|        1|[Portaria 24h, Co...|18d22cbe-1b86-476...|      2|  null|

In [33]:
df_anuncio = df_anuncio\
                    .select('*', 'endereco.bairro', 'endereco.zona')\
                    .drop('endereco')

In [34]:
df_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|Paciência|Zona Oeste|
|    1|        41|       41|        1|[Portaria 24h, Co...|18d22cbe-1b86-476...|      2|  null|

# Tratamento da coluna aninhada de valores

In [35]:
df_anuncio\
    .select([f.col(c)[0].alias(c) if c == 'valores' else c for c in df_anuncio.columns])\
    .show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|{245, null, Venda...|Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|{0, 0, Venda, 15000}|Paciência|Zona Oeste|
|    1|        41|       41|        1|[Portaria 24h, Co...|18d22cbe-1b86-476...|      2|  null|

In [36]:
df_anuncio = df_anuncio\
                .select([f.col(c)[0].alias(c) if c == 'valores' else c for c in df_anuncio.columns])
df_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|{245, null, Venda...|Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|{0, 0, Venda, 15000}|Paciência|Zona Oeste|
|    1|        41|       41|        1|[Portaria 24h, Co...|18d22cbe-1b86-476...|      2|  null|

In [37]:
df_anuncio\
    .select('*', 'valores.*')\
    .drop('valores')\
    .show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|   bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|Paciência|Zona Oeste|         0|   0|Venda|15000|
|    1|        41|       41|        1|[Portaria 24h, Co...|1

In [38]:
df_anuncio = df_anuncio\
                .select('*', 'valores.*')\
                .drop('valores')
df_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|   bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|Paciência|Zona Oeste|         0|   0|Venda|15000|
|    1|        41|       41|        1|[Portaria 24h, Co...|1

# O modelo se tratará de valores de compra de imóveis, sendo assim, serão selecionados somente os imóveis a venda

In [39]:
(df_anuncio\
    .select('tipo')
    .groupby('tipo')
    .count()
    .show())

+-------+-----+
|   tipo|count|
+-------+-----+
|Aluguel|  214|
|  Venda|66348|
+-------+-----+



In [40]:
df_anuncio = df_anuncio\
                .filter("tipo == 'Venda'")
df_anuncio.count()

66348

# Salvando em parquet

In [43]:
df_anuncio.write.parquet(mode = 'overwrite',
                         path = 'dados_transformados/anuncio.parquet')